In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')#change dir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.applications import imagenet_utils
from skimage import data, color, feature

import os
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
from keras.applications.resnet import ResNet101
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet import ResNet152
from keras.applications.inception_v3 import InceptionV3
from keras.applications import MobileNet
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import VGG16

In [ ]:
df_train = pd.read_excel('/content/drive/My Drive/Labels.xlsx')


In [ ]:
df_train.head()

,ImageName,Level
0,fall-01-cam0-rgb-001.png,1
1,fall-01-cam0-rgb-002.png,1
2,fall-01-cam0-rgb-003.png,1
3,fall-01-cam0-rgb-004.png,1
4,fall-01-cam0-rgb-005.png,1


In [ ]:
targets_series = pd.Series(df_train['Level'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [ ]:
one_hot_labels = np.asarray(one_hot)

In [ ]:
im_size1 = 128
im_size2 = 128

In [ ]:
x_train = []
y_train = []
x_test = []

In [ ]:
# i = 0 
# #for folder in range(9):
# for f, breed in tqdm(df_train.values):
#   if type(cv2.imread('IMAGES/{}'.format(f)))==type(None):
#     continue 
#   else: 
#     img = cv2.imread('IMAGES/{}'.format(f)) 
#     label = one_hot_labels[i] 
#     x_train.append(cv2.resize(img, (im_size1, im_size2))) 
#     y_train.append(label) 
#     i += 1 
# np.save('x_trainuURF',x_train) 
# np.save('y_trainURF',y_train) 
# print('Done')

In [ ]:
x_train = np.load('x_trainuURF.npy')
y_train = np.load('y_trainURF.npy')

In [ ]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.

In [ ]:
print(x_train_raw.shape)
print(y_train_raw.shape)

(14161, 128, 128, 3)
(14161, 4)


In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.1, random_state=1)

In [ ]:
num_class = y_train_raw.shape[1]

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.layers import AveragePooling2D
# from keras.layers import AveragePooling
#Optimized ResNet-50

In [ ]:
base_model = ResNet50(weights = None, include_top=False, input_shape=(im_size1, im_size2, 3))

# Add a new top layer
x = base_model.output
x = AveragePooling2D(pool_size=(4, 4))(x)
x = Flatten()(x)
x = Dropout(0.4)(x)
#x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

# model.compile(loss='categorical_crossentropy', 
#               optimizer='RMSprop', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', 
              optimizer='SGD', 
              metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
model.fit(X_train, Y_train, epochs=5, validation_data=(X_valid, Y_valid), verbose=1,shuffle=True)

Epoch 1/5
399/399 [==============================] - 49s 123ms/step - loss: 1.1160 - accuracy: 0.4836 - val_loss: 0.9257 - val_accuracy: 0.6097
Epoch 2/5
399/399 [==============================] - 50s 125ms/step - loss: 0.6217 - accuracy: 0.7445 - val_loss: 0.4339 - val_accuracy: 0.8215
Epoch 3/5
399/399 [==============================] - 49s 124ms/step - loss: 0.3855 - accuracy: 0.8548 - val_loss: 0.4899 - val_accuracy: 0.8144
Epoch 4/5
399/399 [==============================] - 50s 125ms/step - loss: 0.2970 - accuracy: 0.8900 - val_loss: 0.2791 - val_accuracy: 0.8906
Epoch 5/5
399/399 [==============================] - 50s 125ms/step - loss: 0.2352 - accuracy: 0.9149 - val_loss: 0.2281 - val_accuracy: 0.9090


In [ ]:
X_train_features = []
X_valid_features = []

feature_network = Model(base_model.input, model.get_layer('flatten').output)

X_train_features = feature_network.predict(X_train)  # Assuming you have your images in x
X_valid_features = feature_network.predict(X_valid)  # Assuming you have your images in x

print(X_train_features.shape)
print(X_valid_features.shape)
print(Y_train.shape)
print(Y_valid.shape)

(12744, 2048)
(1417, 2048)
(12744, 4)
(1417, 4)


In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import time

# Hyper parameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
svm_orig = svm.LinearSVC(max_iter=1000, dual=False)
svm_orig = GridSearchCV(svm_orig, param_grid)
Y_Train_Array = np.argmax(Y_train, axis=1)
print(Y_Train_Array.shape)

svm_orig.fit(X=X_train_features, y=Y_Train_Array)
print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(svm_orig)

# Predict on test data
svm_predict_orig = svm_orig.predict(X_valid_features)

# Get accuracy
svm_acc_orig = (svm_predict_orig == np.argmax(Y_valid, axis=1)).mean()
print(svm_acc_orig)

(12744,)
--- 416.5655257701874 seconds ---
GridSearchCV(cv=None, error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1.0], 'max_iter': [500, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
0.9795342272406493


In [ ]:
# Linear Kernal Support Vector Machine
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import time

# Hyper parameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
# param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
classifier = SVC(kernel = 'linear', C = 1e-1,  max_iter=1000, random_state = 0)
Y_Train_Array = np.argmax(Y_train, axis=1)
print(Y_Train_Array.shape)


classifier.fit(X=X_train_features, y=Y_Train_Array)
print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(classifier)

# Predict on test data
svm_predict_orig = classifier.predict(X_valid_features)

# Get accuracy
svm_acc_orig = (svm_predict_orig == np.argmax(Y_valid, axis=1)).mean()
print(svm_acc_orig)

(12744,)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


--- 31.132871627807617 seconds ---
SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=1000, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)
0.9760056457304164


In [ ]:
# RBF Support Vector Machine
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import time

# Hyper parameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
# param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
classifier = SVC(kernel = 'rbf', C = 1e-1,  max_iter=1000, random_state = 0)
Y_Train_Array = np.argmax(Y_train, axis=1)
print(Y_Train_Array.shape)


classifier.fit(X=X_train_features, y=Y_Train_Array)
print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(classifier)

# Predict on test data
svm_predict_orig = classifier.predict(X_valid_features)

# Get accuracy
svm_acc_orig = (svm_predict_orig == np.argmax(Y_valid, axis=1)).mean()
print(svm_acc_orig)

(12744,)
--- 67.92956113815308 seconds ---
SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=1000, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)
0.9350741002117149


In [ ]:
# ID3 or Decision Tree classifier 
from sklearn.tree import DecisionTreeClassifier
import time

# Hyper parameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
# param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
Y_Train_Array = np.argmax(Y_train, axis=1)
print(Y_Train_Array.shape)


classifier.fit(X=X_train_features, y=Y_Train_Array)
print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(classifier)

# Predict on test data
svm_predict_orig = classifier.predict(X_valid_features)

# Get accuracy
svm_acc_orig = (svm_predict_orig == np.argmax(Y_valid, axis=1)).mean()
print(svm_acc_orig)

(12744,)
--- 50.57663536071777 seconds ---
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')
0.9456598447424136


In [ ]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
import time

# Hyper parameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
# param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
Y_Train_Array = np.argmax(Y_train, axis=1)
print(Y_Train_Array.shape)


classifier.fit(X=X_train_features, y=Y_Train_Array)
print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(classifier)

# Predict on test data
svm_predict_orig = classifier.predict(X_valid_features)

# Get accuracy
svm_acc_orig = (svm_predict_orig == np.argmax(Y_valid, axis=1)).mean()
print(svm_acc_orig)

(12744,)
--- 8.090665340423584 seconds ---
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
0.9647141848976711


In [ ]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
import time

# Hyper parameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
# param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
classifier = GaussianNB()
Y_Train_Array = np.argmax(Y_train, axis=1)
print(Y_Train_Array.shape)


classifier.fit(X=X_train_features, y=Y_Train_Array)
print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(classifier)

# Predict on test data
svm_predict_orig = classifier.predict(X_valid_features)

# Get accuracy
svm_acc_orig = (svm_predict_orig == np.argmax(Y_valid, axis=1)).mean()
print(svm_acc_orig)

(12744,)
--- 0.19378256797790527 seconds ---
GaussianNB(priors=None, var_smoothing=1e-09)
0.9174311926605505


In [ ]:
# K Neighbors classifier
from sklearn.neighbors import KNeighborsClassifier
import time

# Hyper parameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
# param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
Y_Train_Array = np.argmax(Y_train, axis=1)
print(Y_Train_Array.shape)


classifier.fit(X=X_train_features, y=Y_Train_Array)
print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(classifier)

# Predict on test data
svm_predict_orig = classifier.predict(X_valid_features)

# Get accuracy
svm_acc_orig = (svm_predict_orig == np.argmax(Y_valid, axis=1)).mean()
print(svm_acc_orig)

(12744,)
--- 4.560464859008789 seconds ---
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
0.9738884968242767


In [ ]:
# AdaBoost Classifier
from sklearn.ensemble import AdaBoostClassifier
import time

# Hyper parameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
# param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
classifier = AdaBoostClassifier(n_estimators=50, learning_rate=1)
Y_Train_Array = np.argmax(Y_train, axis=1)
print(Y_Train_Array.shape)


classifier.fit(X=X_train_features, y=Y_Train_Array)
print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(classifier)

# Predict on test data
svm_predict_orig = classifier.predict(X_valid_features)

# Get accuracy
svm_acc_orig = (svm_predict_orig == np.argmax(Y_valid, axis=1)).mean()
print(svm_acc_orig)

(12744,)
--- 187.70654368400574 seconds ---
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
                   n_estimators=50, random_state=None)
0.913196894848271


In [ ]:
# MLP Classifier
from sklearn.neural_network import MLPClassifier
import time
import numpy as np

# Hyper parameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
# param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
classifier = MLPClassifier(max_iter=50, random_state=0)
Y_Train_Array = np.argmax(Y_train, axis=1)
print(Y_Train_Array.shape)


classifier.fit(X=X_train_features, y=Y_Train_Array)
print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(classifier)

# Predict on test data
svm_predict_orig = classifier.predict(X_valid_features)

# Get accuracy
svm_acc_orig = (svm_predict_orig == np.argmax(Y_valid, axis=1)).mean()
print(svm_acc_orig)

(12744,)
--- 47.57319736480713 seconds ---
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=50,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
0.5871559633027523


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
